In [ ]:
!pip install selenium

from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request as req

import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

book_cate = {'소설':'01','시/에세이':'03','인문':'05','경제/경영':'13','자기계발':'15'}
# 01 : 소설 , 03 : 시/에세이 , 05 : 인문 , 13 : 경제/경영 , 15 : 자기계발
# 장르에 따라서 url 설정해야함!!
# book_cate['소설'] 이부분만 바꿔주세용
url = 'https://product.kyobobook.co.kr/category/KOR/'+book_cate['소설']+'#?page=1&type=best&per=20'

driver = webdriver.Chrome()
driver.get(url)

book_titles = list()
book_stories = list()
book_keyPicks = list()

for page in range(1,51):
    time.sleep(3)
    for book in range(1,21):
        xpath = f'/html/body/div[3]/main/section[2]/div/section/div[3]/div[4]/div[3]/ol/li[{book}]/div[2]/div[2]/a'
        book_url = driver.find_element(By.XPATH,xpath)
        book_url.send_keys(Keys.ENTER)
        time.sleep(3)
        title = driver.find_element(By.CSS_SELECTOR,'#contents > div.prod_detail_header > div > div.prod_detail_title_wrap > div > div.prod_title_box.auto_overflow_wrap > div.auto_overflow_contents > div > h1 > span')
        book_titles.append(title.text)
        keywordPick = list()
        try:
            # 키워드 Pick 수집
            keywordPicks = driver.find_elements(By.CSS_SELECTOR,'div.tab_list_wrap > ul >li> a > span.tab_text')
            for keyword in keywordPicks:
                keywordPick.append(keyword.text)
            book_keyPicks.append(keywordPick)
        except:
            pass
        
        try:
            # 책 소개 수집
            story = driver.find_element(By.CSS_SELECTOR,'div.intro_bottom')
            book_stories.append(story.text)
        except:
            pass
        driver.back()
    next_page = driver.find_element(By.CSS_SELECTOR,'#bestBottomPagi > button.btn_page.next')
    next_page.click()
        
driver.quit()


In [ ]:
book_df = pd.DataFrame([book_titles,book_stories,book_keyPicks],index=['상품명','책소개','키워드']).T

def delete_null(keywordList):
    keyword_list = list()
    for x in range(len(keywordList)):
        if keywordList[x] != '':
            keyword_list.append(keywordList[x])
    return keyword_list

book_df['키워드'] = book_df['키워드'].apply(delete_null)

# 파일 이름 바꾸기 위해 book_category[0] 부분만 장르에 맞게 고쳐주세요~!

book_category = ['소설','시/에세이','인문','경제/경영','자기계발']
file_name = book_category[0] + '키워드 Pick.xlsx'
book_df.to_excel(file_name)